In [1]:
#Imported necessary libraries and functions
import numpy as np
import pandas as pd 
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime
%matplotlib inline

In [2]:
#Read all data into dataframes and SQL connection
cast = sqlite3.connect('zippedData/im.db')
df_gross = pd.read_csv("zippedData/bom.movie_gross.csv.gz")
df_info = pd.read_csv("zippedData/rt.movie_info.tsv.gz", sep='\t')
df_reviews = pd.read_csv("zippedData/rt.reviews.tsv.gz", sep='\t', encoding = 'windows-1252')
df_popular = pd.read_csv("zippedData/tmdb.movies.csv.gz")
df_budget = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")

In [3]:
#Converted all movie titles to strings, since some movies were read as integers
df_budget['movie'] = df_budget['movie'].astype(str)

#Removed all special characters from titles and made them all lower case to increase chances of matching other similar values in other databases
spec_char = r'[^\w\s]'
df_budget['movie'].replace(to_replace=spec_char, value='', regex=True, inplace=True)
df_budget['movie'].replace(to_replace=' ', value='', regex=True, inplace=True)
df_budget['movie'] = df_budget['movie'].str.lower()

#Converted release dates to datetime
df_budget['release_date']= pd.to_datetime(df_budget['release_date'])

#Cleaned budget, domestic gross, and worlwide gross columns to integers
df_budget['production_budget'] = df_budget['production_budget'].str.replace(',', '')
df_budget['production_budget'] = df_budget['production_budget'].str.replace('$', '')
df_budget['production_budget'] = df_budget['production_budget'].astype(int)
df_budget['domestic_gross'] = df_budget['domestic_gross'].str.replace(',', '')
df_budget['domestic_gross'] = df_budget['domestic_gross'].str.replace('$', '')
df_budget['domestic_gross'] = df_budget['domestic_gross'].astype(int)
df_budget['worldwide_gross'] = df_budget['worldwide_gross'].str.replace(',', '')
df_budget['worldwide_gross'] = df_budget['worldwide_gross'].str.replace('$', '')
df_budget['worldwide_gross'] = df_budget['worldwide_gross'].astype(np.int64)

df_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,avatar,425000000,760507625,2776345279
1,2,2011-05-20,piratesofthecaribbeanonstrangertides,410600000,241063875,1045663875
2,3,2019-06-07,darkphoenix,350000000,42762350,149762350
3,4,2015-05-01,avengersageofultron,330600000,459005868,1403013963
4,5,2017-12-15,starwarsepviiithelastjedi,317000000,620181382,1316721747


In [4]:
#Added profit columns
df_budget['worldwide_profit'] = df_budget['worldwide_gross'] - df_budget['production_budget']
df_budget['domestic_profit'] = df_budget['domestic_gross'] - df_budget['production_budget']

#Sorted data by highest overall profits worlwide
df_budget.sort_values(by = ['worldwide_profit'], ascending = False, inplace = True)

#Description of numerical data without scientific notation and rounding to the 2nd decimal place
df_budget.describe().apply(lambda s: s.apply('{0:.2f}'.format))

,id,production_budget,domestic_gross,worldwide_gross,worldwide_profit,domestic_profit
count,5782.00,5782.00,5782.00,5782.00,5782.00,5782.00
mean,50.37,31587757.10,41873326.87,91487460.91,59899703.81,10285569.77
std,28.82,41812076.83,68240597.36,174719968.78,146088881.08,49921366.46
min,1.00,1100.00,0.00,0.00,-200237650.00,-307237650.00
25%,25.00,5000000.00,1429534.50,4125414.75,-2189070.75,-9132757.00
50%,50.00,17000000.00,17225945.00,27984448.50,8550285.50,-348775.50
75%,75.00,40000000.00,52348661.50,97645836.50,60968501.75,17781444.00
max,100.00,425000000.00,936662225.00,2776345279.00,2351345279.00,630662225.00


In [5]:
df_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,worldwide_profit,domestic_profit
0,1,2009-12-18,avatar,425000000,760507625,2776345279,2351345279,335507625
42,43,1997-12-19,titanic,200000000,659363944,2208208395,2008208395,459363944
6,7,2018-04-27,avengersinfinitywar,300000000,678815482,2048134200,1748134200,378815482
5,6,2015-12-18,starwarsepviitheforceawakens,306000000,936662225,2053311220,1747311220,630662225
33,34,2015-06-12,jurassicworld,215000000,652270625,1648854864,1433854864,437270625


In [6]:
%%bash 

sqlite3 zippedData/im.db
.tables

directors      movie_akas     movie_ratings  principals   
known_for      movie_basics   persons        writers      


In [7]:
#Created a query that extracted the necessary information from the im.db file
review_query = """
    SELECT movie_basics.primary_title AS title, movie_basics.genres AS genres, 
    movie_ratings.averagerating AS average_rating, movie_ratings.numvotes AS num_votes, 
    movie_akas.language AS language, movie_akas.region AS region
    FROM movie_basics
    JOIN movie_ratings
        ON movie_basics.movie_id = movie_ratings.movie_id
    JOIN movie_akas
        ON movie_basics.movie_id = movie_akas.movie_id
    ORDER BY movie_ratings.averagerating DESC
"""
#Removed WHERE language = 'en' because there is an enormous amount of missing data

#Converted SQL query to Pandas Dataframe
df_reviews = pd.read_sql(review_query, cast)

#Removed all special characters from titles and made them all lower case to increase chances of matching other similar values in other databases
df_reviews['title'].replace(to_replace=spec_char, value='', regex=True, inplace=True)
df_reviews['title'].replace(to_replace=' ', value='', regex=True, inplace=True)
df_reviews['title'] = df_reviews['title'].str.lower()

#Replaced NaNs in genres column with a placeholder string so it can be iterable when converted to a dictionary
df_reviews['genres'] = df_reviews['genres'].fillna('Unknown')
df_reviews.head()

,title,genres,average_rating,num_votes,language,region
0,exterioresmulheresbrasileirasnadiplomacia,Documentary,10.0,5,None,None
1,exterioresmulheresbrasileirasnadiplomacia,Documentary,10.0,5,None,BR
2,exterioresmulheresbrasileirasnadiplomacia,Documentary,10.0,5,en,XWW
3,freeingberniebaran,"Crime,Documentary",10.0,5,None,US
4,herculecontrehermès,Documentary,10.0,5,None,FR


In [8]:
#Created dictionary with movie titles as keys and list of genres as values
genres = pd.Series(df_reviews.genres.values, index = df_reviews.title).to_dict()
delimiter = ','
for key, value in genres.items():
    genres[key] = value.split(delimiter)
genres

{'exterioresmulheresbrasileirasnadiplomacia': ['Documentary'],
 'freeingberniebaran': ['Crime', 'Documentary'],
 'herculecontrehermès': ['Documentary'],
 'iwasbornyesterday': ['Documentary'],
 'dogdaysintheheartland': ['Drama'],
 'revolutionfood': ['Documentary'],
 'flyhighstoryofthediscdog': ['Documentary'],
 'allaroundus': ['Documentary'],
 'thepaternalbondbarbarymacaques': ['Documentary'],
 'requiemvooreenboom': ['Documentary'],
 'adedicatedlifephoebebrandbeyondthegroup': ['Documentary'],
 'ellisislandthemakingofamasterraceinamerica': ['Documentary', 'History'],
 'calamitykevin': ['Adventure', 'Comedy'],
 'mujeresrepublicanas': ['Documentary'],
 'sendmymailtonashville': ['Biography', 'Documentary', 'Music'],
 'maasai10thlosttribeofisrael': ['Biography', 'Documentary', 'Drama'],
 'ourfriendjon': ['Documentary'],
 'hirostable': ['Documentary'],
 'piecesoflivespiecesofdreams': ['Documentary'],
 'laleccióndeanatomía': ['Documentary'],
 'hizam': ['Documentary'],
 'goreciskof': ['Document

In [9]:
#Created a list of genre values
list_of_genres = []
for key, value in genres.items():
    for index in value:
        if index not in list_of_genres:
            list_of_genres.append(index)
list_of_genres

['Documentary',
 'Crime',
 'Drama',
 'History',
 'Adventure',
 'Comedy',
 'Biography',
 'Music',
 'Family',
 'Romance',
 'Sport',
 'War',
 'Animation',
 'Musical',
 'Horror',
 'Mystery',
 'Thriller',
 'Action',
 'Fantasy',
 'News',
 'Unknown',
 'Sci-Fi',
 'Game-Show',
 'Western',
 'Reality-TV',
 'Adult']

In [21]:
#Counted the number of films in each genre
genre_count = {}
for genre in list_of_genres:
    genre_count[genre] = 0
for key, value in genres.items():
    for index in value:
        genre_count[index] += 1
genre_count

{'Documentary': 15927,
 'Crime': 4149,
 'Drama': 27288,
 'History': 2628,
 'Adventure': 3542,
 'Comedy': 15703,
 'Biography': 3561,
 'Music': 1818,
 'Family': 3086,
 'Romance': 5887,
 'Sport': 1041,
 'War': 784,
 'Animation': 1603,
 'Musical': 639,
 'Horror': 6882,
 'Mystery': 2712,
 'Thriller': 7243,
 'Action': 6333,
 'Fantasy': 1957,
 'News': 555,
 'Unknown': 602,
 'Sci-Fi': 2014,
 'Game-Show': 2,
 'Western': 253,
 'Reality-TV': 16,
 'Adult': 3}

In [10]:
# Perform inner join on 'B' column in df1 and 'D' column in df2
result = pd.merge(df_budget, df_reviews, how='inner', left_on='movie', right_on='title')

# Extract the values that are in both columns
common_values = result['movie'].tolist()
movie_titles = list(set(common_values))
print(len(movie_titles))

2219


In [11]:
#Created a new dataframe that had combined, cleaned, and index-aligned data on worldwide/domestic profit, average rating, release date, genres, and number of votes
worldwide_profit = []
ww = 0
domestic_profit = []
dom = 0
average_rating = []
rat = 0.0
release_date = []
rel = None
movie_genres = []
gen = None
vote_count = []
vote = None

for title in movie_titles:
    ww = df_budget.loc[df_budget['movie'] == title, 'worldwide_profit'].iloc[0]
    worldwide_profit.append(ww)
    dom = df_budget.loc[df_budget['movie'] == title, 'domestic_profit'].iloc[0]
    domestic_profit.append(dom)
    rat = df_reviews.loc[df_reviews['title'] == title, 'average_rating'].iloc[0]
    average_rating.append(rat)
    rel = df_budget.loc[df_budget['movie'] == title, 'release_date'].iloc[0].date()
    release_date.append(rel)
    gen = df_reviews.loc[df_reviews['title'] == title, 'genres'].iloc[0]
    movie_genres.append(gen)
    vote = df_reviews.loc[df_reviews['title'] == title, 'num_votes'].iloc[0]
    vote_count.append(vote)

d = {'title': movie_titles, 'release_date': release_date, 'movie_genres': movie_genres, 'worldwide_profit': worldwide_profit, 'domestic_profit': domestic_profit, 'average_rating': average_rating, 'num_votes': vote_count}
df = pd.DataFrame(data = d)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2219 entries, 0 to 2218
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             2219 non-null   object 
 1   release_date      2219 non-null   object 
 2   movie_genres      2219 non-null   object 
 3   worldwide_profit  2219 non-null   int64  
 4   domestic_profit   2219 non-null   int64  
 5   average_rating    2219 non-null   float64
 6   num_votes         2219 non-null   int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 121.5+ KB


In [14]:
#Ordered by worldwide profit
df_ww = df.sort_values(by = 'worldwide_profit', ascending = False)
df_ww.head(10)

,title,release_date,movie_genres,worldwide_profit,domestic_profit,average_rating,num_votes
1564,avatar,2009-12-18,Horror,2351345279,335507625,6.1,43
1094,titanic,1997-12-19,Adventure,2008208395,459363944,6.2,20
405,avengersinfinitywar,2018-04-27,"Action,Adventure,Sci-Fi",1748134200,378815482,8.5,670926
2164,jurassicworld,2015-06-12,"Action,Adventure,Sci-Fi",1433854864,437270625,7.0,539338
71,furious7,2015-04-03,"Action,Crime,Thriller",1328722794,163007020,7.2,335074
992,theavengers,2012-05-04,"Action,Adventure,Sci-Fi",1292935897,398279547,8.1,1183655
807,blackpanther,2018-02-16,"Action,Adventure,Sci-Fi",1148258224,500059566,7.3,516148
2134,jurassicworldfallenkingdom,2018-06-22,"Action,Adventure,Sci-Fi",1135772799,247719760,6.2,219125
1923,frozen,2013-11-22,"Adventure,Animation,Comedy",1122469910,250738009,7.5,516998
325,beautyandthebeast,2017-03-17,"Family,Fantasy,Musical",1099199706,344014165,7.2,238325


In [17]:
#Ordered by Domestic Profit
df_dom = df.sort_values(by = 'domestic_profit', ascending = False)
df_dom.head(10)

,title,release_date,movie_genres,worldwide_profit,domestic_profit,average_rating,num_votes
807,blackpanther,2018-02-16,"Action,Adventure,Sci-Fi",1148258224,500059566,7.3,516148
1094,titanic,1997-12-19,Adventure,2008208395,459363944,6.2,20
2164,jurassicworld,2015-06-12,"Action,Adventure,Sci-Fi",1433854864,437270625,7.0,539338
955,incredibles2,2018-06-15,"Action,Adventure,Animation",1042520711,408581744,7.7,203510
992,theavengers,2012-05-04,"Action,Adventure,Sci-Fi",1292935897,398279547,8.1,1183655
405,avengersinfinitywar,2018-04-27,"Action,Adventure,Sci-Fi",1748134200,378815482,8.5,670926
325,beautyandthebeast,2017-03-17,"Family,Fantasy,Musical",1099199706,344014165,7.2,238325
1564,avatar,2009-12-18,Horror,2351345279,335507625,6.1,43
515,rogueoneastarwarsstory,2016-12-16,"Action,Adventure,Sci-Fi",849102856,332177324,7.8,478592
1705,thehungergames,2012-03-23,"Action,Adventure,Sci-Fi",597923379,328010692,7.2,795227


In [20]:
#Ordered by rating
df_rat = df.sort_values(by = 'average_rating', ascending = False)
df_rat = df_rat.loc[df_rat['num_votes'] >= 5]
df_rat.head(10)

,title,release_date,movie_genres,worldwide_profit,domestic_profit,average_rating,num_votes
1853,asgoodasitgets,1997-12-24,Documentary,264111923,98478011,9.4,60
1205,therunaways,2010-03-19,Adventure,-4221368,-5926327,9.2,47
1751,thewall,2017-05-12,Documentary,1495262,-1196936,9.2,8
1149,traffic,2000-12-27,Documentary,160300000,76107476,9.2,64
1776,waiting,2005-10-07,Unknown,17548274,14999543,9.1,82
147,5050,2011-09-30,Documentary,33334735,27016118,9.0,10
1485,dragonfly,2002-02-22,"Action,Adventure,Sci-Fi",-29936195,-29936195,9.0,5
633,survivor,2015-05-29,"Biography,Crime,Documentary",-18296719,-20000000,9.0,45
1460,frailty,2002-04-12,Drama,8947280,2110448,9.0,13
708,frankenstein,1994-11-04,Drama,67006296,-22993704,9.0,1832


In [35]:
#Calculated the total average profit gained by each genre
genre_profits = {}
for genre in list_of_genres:
    total_worldwide_profit = 0
    n_ww = 0
    total_domestic_profit = 0
    n_dom = 0
    for i in range(len(df['title'])):
        if genre in df['movie_genres'][i]:
            total_worldwide_profit += df['worldwide_profit'][i]
            total_domestic_profit += df['domestic_profit'][i]
            n_ww += 1
            n_dom += 1
    if total_worldwide_profit > 0:
        genre_ww_profit = round((total_worldwide_profit / n_ww), 2)
        genre_dom_profit = round((total_domestic_profit / n_dom), 2)
        genre_profits[genre] = (genre_ww_profit, genre_dom_profit)

genre_profits

{'Documentary': (28116866.51, 3862614.63),
 'Crime': (40422703.96, 3211250.63),
 'Drama': (38628852.92, 6089159.07),
 'History': (31343805.95, 351691.36),
 'Adventure': (226614685.97, 19954770.95),
 'Comedy': (77419724.68, 15150036.85),
 'Biography': (48134316.0, 10392999.03),
 'Music': (59089457.56, 17198240.7),
 'Family': (108466625.66, 14188169.39),
 'Romance': (38299943.71, 8057940.95),
 'Sport': (17086400.24, 4802805.75),
 'War': (23037724.35, -1978462.42),
 'Animation': (237312062.49, 30122930.19),
 'Musical': (81623352.21, 19311892.0),
 'Horror': (49828250.89, 12274095.34),
 'Mystery': (53322101.36, 10785225.29),
 'Thriller': (65686684.33, 8201380.28),
 'Action': (145853602.7, 9257602.26),
 'Fantasy': (143254163.82, 1819892.61),
 'News': (20082077.0, -7196259.0),
 'Unknown': (55445528.25, -16635568.25),
 'Sci-Fi': (201994073.29, 23483445.13),
 'Western': (27861739.35, -12089071.47)}